In [2]:
import torch
from models import skipgram
import pandas as pd
import wandb

run = wandb.init(project="word2vec")
artifact = run.use_artifact("model-weights:latest")
datadir = artifact.download()

MINI MODE: True


wandb: Currently logged in as: cameronnimmo (cnimmo16) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


wandb:   1 of 1 files downloaded.  


In [7]:

model = skipgram.Model(vocab.size + 1, skipgram.EMBEDDING_DIM)

state_dict = torch.load('./data/weights.generated.pt')
# model.load_state_dict(weights)
# model.eval()

print(state_dict)


OrderedDict({'embeddings.weight': tensor([[ 1.9269,  1.4873,  0.9007,  ..., -0.4879, -0.9138, -0.6581],
        [ 0.0780,  0.5258, -0.4880,  ...,  0.4880,  0.7846,  0.0286],
        [ 0.7921,  0.7724,  1.2230,  ...,  1.6339,  0.4167, -0.3995],
        ...,
        [-0.6330,  0.1846, -0.8211,  ...,  0.2991,  0.7049,  0.8997],
        [-0.9033, -1.2141, -0.7326,  ..., -1.3640,  0.3009,  0.7943],
        [-1.0489, -0.5225, -1.1338,  ...,  0.8321,  0.9070,  0.0391]]), 'linear.weight': tensor([[-0.0882,  0.0111, -0.0100,  ..., -0.0461, -0.0593, -0.0729],
        [ 0.2270,  0.0895,  0.2126,  ...,  0.2022,  0.1374, -0.0352],
        [-0.0056,  0.1953,  0.2204,  ...,  0.0697,  0.0889,  0.0642],
        ...,
        [-0.2915, -0.0183,  0.1372,  ...,  0.0169,  0.0151,  0.1846],
        [ 0.0025,  0.2679, -0.0650,  ...,  0.2539,  0.0110,  0.0057],
        [-0.2341, -0.0694, -0.2223,  ..., -0.0881,  0.0627,  0.0505]]), 'linear.bias': tensor([ 0.0992, -0.0737, -0.0629,  0.1086,  0.0553, -0.0804,  0

In [40]:
vocab = pd.read_csv(f"data/vocab{'__mini' if skipgram.MINIMODE else ''}.generated.csv", index_col='token')

def getIdFromToken(token: str):
    try:
        return int(vocab.at[token, 'id'])
    except:
        return int(vocab.at['unk', 'id'])

#find the n most similar words
def most_similar(word, n=5):
    word_idx = getIdFromToken(word)
    A =  state_dict['embeddings.weight'][word_idx].unsqueeze(0)
    word_similarities = []
    for token, ids in vocab.iterrows():
        id = ids.values[0]
        B =  model.embeddings.weight[id].unsqueeze(0)
        cosine_similarity = torch.nn.functional.cosine_similarity(A, B, dim=1)
        word_similarities.append((token, cosine_similarity.item()))
    word_similarities = sorted(word_similarities, key=lambda x: x[1], reverse=True)
    return word_similarities[:n]

most_similar('ml')

[('ml', 0.38398277759552),
 ('decisions', 0.3758777678012848),
 ('models', 0.19850623607635498),
 ('improve', 0.13952332735061646),
 ('unk', 0.10921022295951843)]